In [ ]:
import itertools
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from handcrafted.app.dataset.dataset_loader import DatasetLoader
from handcrafted.app.dataset.utils.dataset_creator import DatasetCreator
from handcrafted.app.dataset.utils.frames_splitter import FramesSplitter
from handcrafted.app.model.model_statistics import ModelStatistics

In [ ]:
loader = DatasetLoader(directory="./data/frames_no_bg/")

In [ ]:
num_signers = 8
if num_signers == -1:
    signers = loader.signers
    num_signers = loader.num_signers
else:
    signers = dict(itertools.islice(loader.signers.items(), num_signers))
splitter = FramesSplitter(signers, val_split=0.3, test_split=0.3, frames_split=0.1, seed=42)

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = splitter.split(X_content=lambda f: f)

In [ ]:
# Get the total of different labels
num_classes = num_signers
print(num_classes)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
BATCH_SIZE = 512
dataset_creator = DatasetCreator()
# TODO: add shuffle = True
train_dataset = dataset_creator.create_custom_dataset(X_train, y_train_encoded, batch_size=BATCH_SIZE)
val_dataset = dataset_creator.create_custom_dataset(X_val, y_val_encoded, batch_size=BATCH_SIZE)
test_dataset = dataset_creator.create_custom_dataset(X_test, y_test_encoded, batch_size=BATCH_SIZE)


In [ ]:
xgb_params = {
    "objective":"multi:softmax",
    "num_class":num_classes,
    "eval_metric":"mlogloss",
    "max_depth":6,
    "learning_rate":0.1,
    "n_estimators":100,
    "subsample":0.8,
    "colsample_bytree":0.8,
    "gamma":0,
    "min_child_weight":1,
    "reg_lambda":1,
    "reg_alpha":0,
    "tree_method":"hist", # Change to "gpu_hist" for GPU acceleration
    # "device":"cuda", # Use GPU
}

In [ ]:
xgb_model = None
for train_batch in tqdm(val_dataset):
    X_batch, y_batch = train_batch.load(num_aug=3, shuffle=True)
    dtrain = xgb.DMatrix(X_batch, label=y_batch)
    xgb_model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=10, xgb_model=xgb_model)

In [ ]:
y_pred = []
y_test_full = []
for test_batch in tqdm(test_dataset):
    X_batch, y_batch = test_batch.load(num_aug=0, shuffle=True)
    y_test_full.extend(y_batch)
    dtest = xgb.DMatrix(X_batch)
    y_pred_batch = xgb_model.predict(dtest)
    y_pred.extend(y_pred_batch)

In [ ]:
stats = ModelStatistics(save_name=f"svc_signer_test_{len(y_test_full)}", save_dir="signer/plots")
stats.print_accuracy(y_test_full, y_pred)
stats.plot_confusion_matrix(y_test_full, y_pred, save=True, plot=True)